In [ ]:
include("atlas_option.jl")

In [63]:
cov_matrix = [1.0 0.3 0.4; 0.3 1.0 0.1; 0.4 0.1 1.0] 
normal = [atlas_option_monte_carlo(10000,0.05,10,0.003,7.0,3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.01,0.01,0.01],cov_matrix,1,1,"basic") for i in 1:2]
antithetic = [atlas_option_monte_carlo(10000,0.05,10,0.003,7.0,3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.01,0.01,0.01],cov_matrix,1,1,"antithetic") for i in 1:2]
quasi = [atlas_option_monte_carlo(10000,0.05,10,0.003,7.0,3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.01,0.01,0.01],cov_matrix,1,1,"quasi_monte_carlo") for i in 1:2]
moment_matching = [price_atlas_moment_matching(10000,0.05,10,0.003,7.0,3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.01,0.01,0.01],cov_matrix,1,1) for i in 1:2]
LHS = [atlas_option_monte_carlo(10000,0.05,10,0.003,7.0,3,[10.0,10.0,10.0],[0.07,0.07,0.07],[0.01,0.01,0.01],cov_matrix,1,1,"LHS") for i in 1:2]

mean_elements = mean([normal[i][1] for i in eachindex(normal)])
bottom_elements = mean([normal[i][2] for i in eachindex(normal)])
upper_elements = mean([normal[i][3] for i in eachindex(normal)])

mean_antithetic = mean([antithetic[i][1] for i in eachindex(antithetic)])
bottom_antithetic = mean([antithetic[i][2] for i in eachindex(antithetic)])
upper_antithetic = mean([antithetic[i][3] for i in eachindex(antithetic)])

mean_quasi = mean([quasi[i][1] for i in eachindex(quasi)])
bottom_quasi = mean([quasi[i][2] for i in eachindex(quasi)])
upper_quasi = mean([quasi[i][3] for i in eachindex(quasi)])

mean_moment_matching = mean([moment_matching[i][1] for i in eachindex(moment_matching)])
bottom_moment_matching = mean([moment_matching[i][2] for i in eachindex(moment_matching)])
upper_moment_matching = mean([moment_matching[i][3] for i in eachindex(moment_matching)])

mean_LHS = mean([LHS[i][1] for i in eachindex(LHS)])
bottom_LHS = mean([LHS[i][2] for i in eachindex(LHS)])
upper_LHS = mean([LHS[i][3] for i in eachindex(LHS)])

length_normal = upper_elements-bottom_elements
length_antithetic = upper_antithetic - bottom_antithetic
length_quasi = upper_quasi-bottom_quasi
length_moment_matching = upper_moment_matching - bottom_moment_matching
length_LHS = upper_LHS-bottom_LHS

println("basic mean : $mean_elements length: $length_normal")
println("antithetic mean : $mean_antithetic length antithetic: $length_antithetic")
println("quasi mean : $mean_quasi length quasi: $length_quasi")
println("moment matching mean : $mean_moment_matching length matching: $length_moment_matching")
println("LHS mean : $mean_LHS length LHS: $length_LHS")


In [45]:
mean_moment_matching

3.0004516866860795